In [19]:
import kfp

In [7]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")

In [8]:
storage_op = kfp.components.load_component_from_file("/mnt/dkube/pipeline/components/storage/component.yaml")

In [16]:
@kfp.dsl.pipeline(name='StorageOp-test',description='StorageOp component')

def storageop_testcases(auth_token,dataset_name,model_name,dataset_version,model_version):
    with kfp.dsl.ExitHandler(exit_op=storage_op("reclaim", auth_token, namespace="kubeflow",uid="{{workflow.uid}}")):
            input_volumes = json.dumps(["{{workflow.uid}}-dataset@dataset://"+str(dataset_name)+"/"+str(dataset_version),"{{workflow.uid}}-model@model://" + str(model_name)+"/"+str(model_version)])
            output_volumes=json.dumps(["{{workflow.uid}}-dataset@dataset://"+str(dataset_name),"{{workflow.uid}}-model@model://" + str(model_name)])
            storage  = storage_op("export", token, namespace="kubeflow", input_volumes=input_volumes, output_volumes = output_volumes)
            train = kfp.dsl.ContainerOp(
                name="container-op",
                image="docker.io/ocdr/dkube-datascience-tf-cpu:v2.0.0-3",
                command="bash", 
                arguments=["-c", "ls /model"],
                pvolumes={
                         "/dataset": kfp.dsl.PipelineVolume(pvc="{{workflow.uid}}-dataset"),
                         "/model": kfp.dsl.PipelineVolume(pvc="{{workflow.uid}}-model")
                         }).after(storage)
            

In [17]:
dataset_name = "arv-s3"
model_name = "Img-DN"
dataset_version = "v1"
model_version = "v2"

In [18]:
client = kfp.Client(existing_token=token)
client.create_run_from_pipeline_func(storageop_testcases,arguments={"auth_token":token,"dataset_name":dataset_name,"model_name":model_name,"dataset_version":dataset_version,"model_version":model_version})



RunPipelineResult(run_id=8c610e4f-dd90-44b4-90ff-16e1ebd323bd)